In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Installation

In [ ]:
!pip -q install rouge_score

  Preparing metadata (setup.py) ... done


In [ ]:
!pip -q install langchain langchain_openai PyPDF2 faiss-cpu evaluate torch

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 867.6/867.6 kB 11.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 12.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.0/27.0 MB 61.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 12.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 86.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.8/302.8 kB 36.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.7/116.7 kB 16.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 312.9/312.9 kB 35.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 33.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 56.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 18.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 27.1 MB/s eta 0:00:00
     ━━━━━━━━━━━

In [ ]:
!pip install bitsandbytes

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.8/119.8 MB 13.4 MB/s eta 0:00:00


In [ ]:
pip install cohere

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 151.2/151.2 kB 4.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 20.4 MB/s eta 0:00:00


In [ ]:
!pip install accelerate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.4/302.4 kB 6.9 MB/s eta 0:00:00


# Read the PDF

In [ ]:
import os
import torch
import transformers
import evaluate
import json
import openai

from langchain_openai.chat_models import ChatOpenAI
from langchain_openai.embeddings import OpenAIEmbeddings
from PyPDF2 import PdfReader
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import FAISS
from langchain.chains.question_answering import load_qa_chain
from langchain.llms import OpenAI
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
from langchain.chains import RetrievalQA
from langchain.retrievers import ContextualCompressionRetriever
from langchain.retrievers.document_compressors import CohereRerank
from datasets import load_dataset, load_metric
from pathlib import Path
from openai import OpenAI

In [ ]:
doc_reader = PdfReader('/content/drive/Shareddrives/Data255_Lab/Lab2/part3/1Q23-EPR-with-Tables-FINAL.pdf')
doc_reader

In [ ]:
raw_text = ''
for i, page in enumerate(doc_reader.pages):
    text = page.extract_text()
    if text:
        raw_text += text

In [ ]:
# The count of words
len(raw_text)

33762

In [ ]:
# Split the text
text_splitter = CharacterTextSplitter(
    separator = "\n",
    chunk_size = 1000,
    chunk_overlap  = 200, #striding over the text
    length_function = len,
)
texts = text_splitter.split_text(raw_text)

In [ ]:
len(texts)

44

In [ ]:
texts[0]

'Gap Inc. Reports First  Quarter Fiscal 2023 Results   \n \nSAN FRANCISCO – May 25 , 202 3 – Gap Inc . (NYSE: GPS), a portfolio of purpose -led, billion -dollar lifestyle  \nbrands including Old Navy, Gap, Banana Republic, and Athleta, and the largest specialty apparel \ncompany in the U.S ., today reported financial results for its first  quarter ended April 29, 202 3.  \n “We continue to take the necessary actions to drive critical change at Gap Inc., ultimately getting us \nback on a path toward delivering consistent results long -term,” said Bob Martin, Executive Chairman \nand Interim CEO, Gap Inc. “While the macro and consum er environment remain uncertain, Q1 \nunderscores our ability to deliver improvements to the business including share gains at Old Navy and \nGap Brand, adjusted operating margin expansion, reduction in inventory, and strength in our balance \nsheet. The need fo r lasting change is permeating the organization and I want to express my gratitude to'

# Vector DB

In [ ]:


model = ChatOpenAI(api_key = OPENAI_API_KEY, model = "gpt-3.5-turbo")
model.invoke("Hello")

AIMessage(content='Hello! How can I assist you today?', response_metadata={'token_usage': {'completion_tokens': 9, 'prompt_tokens': 8, 'total_tokens': 17}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': 'fp_a450710239', 'finish_reason': 'stop', 'logprobs': None}, id='run-12fa6e49-2095-4999-be01-47d2339277cb-0')

In [ ]:
embeddings = OpenAIEmbeddings(api_key = OPENAI_API_KEY)

vectorstore = FAISS.from_texts(texts, embeddings)

# RAG

In [ ]:
retriever = vectorstore.as_retriever(search_kwargs={"k": 20})

In [ ]:
compressor = CohereRerank(cohere_api_key = 'cV3IHRjU3gHNvbmyLkiUU8AT4FVrGKjloelpKhnH')
compression_retriever = ContextualCompressionRetriever(
    base_compressor=compressor,
    base_retriever=retriever
)

In [ ]:
rag_pipeline = RetrievalQA.from_chain_type(llm=model,
                                 chain_type="stuff",
                                 retriever=compression_retriever)

In [ ]:
query_set = [
    "A company produces 500 tons of plastic packaging annually and is subject to an Extended Producer Responsibility (EPR) scheme with a recovery target of 80%. How many tons of plastic packaging should the company aim to recover each year to meet the EPR requirement?",
    "What is the report quarter and when did it end?",
]

predictions = []

for query in query_set:
    predictions.append(rag_pipeline.run(query))
    print(rag_pipeline.run(query))

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


To meet the Extended Producer Responsibility (EPR) scheme's recovery target of 80%, the company should aim to recover 80% of the 500 tons of plastic packaging it produces annually.

Calculating 80% of 500 tons:
500 tons * 0.80 = 400 tons

Therefore, the company should aim to recover 400 tons of plastic packaging each year to meet the EPR requirement.
The report quarter mentioned is the first quarter of fiscal 2023, and it ended on April 29, 2023.


In [ ]:
# Load the ROUGE metric
rouge = load_metric('rouge')

references = [
    ["EPR Recovery Target Calculation: To meet the recovery target of 80%, the company should aim to recover 80% 80% of 500 tons, which is: Recovery Target=80%×500 tons=0.80×500 tons=400 tons Recovery Target=80%×500tons=0.80×500tons=400tons So, the company should aim to recover 400 tons of plastic packaging each year."],
    ["The reported quarter is the first financial quarter of 2023. The quarter ended on April 29, 2023"]
]

# Compute ROUGE scores
results = rouge.compute(predictions=predictions, references=references)
results

/usr/local/lib/python3.10/dist-packages/datasets/load.py:759: FutureWarning: The repository for rouge contains custom code which must be executed to correctly load the metric. You can inspect the repository content at https://raw.githubusercontent.com/huggingface/datasets/2.19.0/metrics/rouge/rouge.py
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this metric from the next major release of `datasets`.
  warnings.warn(


{'rouge1': AggregateScore(low=Score(precision=0.5849056603773585, recall=0.543859649122807, fmeasure=0.5636363636363636), mid=Score(precision=0.6674528301886793, recall=0.6248710010319918, fmeasure=0.6454545454545454), high=Score(precision=0.75, recall=0.7058823529411765, fmeasure=0.7272727272727272)),
 'rouge2': AggregateScore(low=Score(precision=0.4, recall=0.375, fmeasure=0.38709677419354843), mid=Score(precision=0.4115384615384615, recall=0.3839285714285714, fmeasure=0.3972520908004779), high=Score(precision=0.4230769230769231, recall=0.39285714285714285, fmeasure=0.4074074074074074)),
 'rougeL': AggregateScore(low=Score(precision=0.4339622641509434, recall=0.40350877192982454, fmeasure=0.41818181818181815), mid=Score(precision=0.5919811320754718, recall=0.5546955624355006, fmeasure=0.5727272727272726), high=Score(precision=0.75, recall=0.7058823529411765, fmeasure=0.7272727272727272)),
 'rougeLsum': AggregateScore(low=Score(precision=0.4339622641509434, recall=0.40350877192982454,

In [ ]:
bleu = evaluate.load('bleu')

references = [
    ["EPR Recovery Target Calculation: To meet the recovery target of 80%, the company should aim to recover 80% 80% of 500 tons, which is: Recovery Target=80%×500 tons=0.80×500 tons=400 tons Recovery Target=80%×500tons=0.80×500tons=400tons So, the company should aim to recover 400 tons of plastic packaging each year."],
    ["The reported quarter is the first financial quarter of 2023. The quarter ended on April 29, 2023"]
]

results = bleu.compute(predictions=predictions, references=references,
          max_order = 2)
results

{'bleu': 0.5693457555580621,
 'precisions': [0.6630434782608695, 0.4888888888888889],
 'brevity_penalty': 1.0,
 'length_ratio': 1.0337078651685394,
 'translation_length': 92,
 'reference_length': 89}

# Fine-tuning

In [ ]:
query = "A company produces 500 tons of plastic packaging annually and is subject to an Extended Producer Responsibility (EPR) scheme with a recovery target of 80%. How many tons of plastic packaging should the company aim to recover each year to meet the EPR requirement?"

documents = vectorstore.similarity_search(query)
content_1 = ""

for document in documents:
    content_1 += document.page_content + "\n"

print(content_1)

managing inventory and integrating pack and hold inventory in fiscal 2023; expected ending inventory in 
fiscal 2023; expected cash flow trends through fiscal 2023; paying down our asset -backed line of credit 
in fiscal 2023 ; our dividend strategy; the expected impact of the Gap China transition, foreign exchange 
headwinds and an additional fiscal week on second quarter and fiscal 2023 net sales; expected second 
quarter and fiscal 2023 net sales; expected second quarter and f iscal 2023 adjusted gross margin; 
expected gross margin improvement in fiscal 2023; expected air freight expense in the second quarter and fiscal 2023; expected inflationary impacts, including commodity costs and ocean freight rates, in the second quarter and fiscal 2023; expected inventory positions  and promotional  activity in the second 
quarter and fiscal 2023; expected ROD in the second quarter and fiscal 2023; expected adjusted
on our financial condition, results of operations, and reputation: the over

In [ ]:
query = "What is the report quarter and when did it end?"

documents = vectorstore.similarity_search(query)
content_2 = ""

for document in documents:
    content_2 += document.page_content + "\n"

print(content_2)

• The effective tax rate was 10 %.  
• Reported net loss  of $18 million ; reported diluted loss  per share of $ 0.05.  
• Adjusted net income of $ 3 million , excluding the gain on sale and restructuring costs ; adjusted 
diluted earnings per share of $0.01.  
 
F
irst Quarter Fiscal 202 3 – Balance Sheet  and Cash Flow Highlights  
 
• Ended the quarter with  cash and cash equivalents of $1.2 billion , an increase of 38% from the 
prior year. 
• Net cash from operating activities was $15 million. F ree cash flow , defined as net cash from 
operating activities less purchases of property and equipment, was negative  $102 million.  
• Ending inventory of $2.3 billion was down 27% compared  to last  year .  
• Capital expenditures were $117 million. 
• Paid first  quarter dividend of $0.15 per share, totaling $ 55 million . Board of Directors approved 
second quarter fiscal 202 3 dividend of $0.15 per share.
Gap Inc. Reports First  Quarter Fiscal 2023 Results   
 
SAN FRANCISCO – May 25

In [ ]:
query = "If a municipality introduces a pay-as-you-throw program as part of its EPR strategy, charging households $0.05 per pound of collected waste, calculate the cost for a household that generates 30 pounds per week."

documents = vectorstore.similarity_search(query)
content_3 = ""

for document in documents:
    content_3 += document.page_content + "\n"

query = "What's the OP margin for the current quarter?"

documents = vectorstore.similarity_search(query)
content_4 = ""

for document in documents:
    content_4 += document.page_content + "\n"

query = "What were Gap Inc's net sales for the first quarter of fiscal 2023 and how did they compare to the previous year?"

documents = vectorstore.similarity_search(query)
content_5 = ""

for document in documents:
    content_5 += document.page_content + "\n"

query = "What is the company's outlook on net sales for the second quarter and the full fiscal year 2023?"

documents = vectorstore.similarity_search(query)
content_6 = ""

for document in documents:
    content_6 += document.page_content + "\n"

query = "How much cash and cash equivalents did Gap, Inc. have at the end of the quarter?"

documents = vectorstore.similarity_search(query)
content_7 = ""

for document in documents:
    content_7 += document.page_content + "\n"

query = "How does the company expect gross margin to change in Q2 and the full year compared to 2022?"

documents = vectorstore.similarity_search(query)
content_8 = ""

for document in documents:
    content_8 += document.page_content + "\n"

query = "Did Gap Inc. pay a dividend in the first quarter, and what is their dividend strategy going forward?"

documents = vectorstore.similarity_search(query)
content_9 = ""

for document in documents:
    content_9 += document.page_content + "\n"

query = "Did Gap Inc. have an operating profit or loss, and if so, what was the amount?"

documents = vectorstore.similarity_search(query)
content_10 = ""

for document in documents:
    content_10 += document.page_content + "\n"

In [ ]:
data = [
    {
        "context": content_1,
        "question": "A company produces 500 tons of plastic packaging annually and is subject to an Extended Producer Responsibility (EPR) scheme with a recovery target of 80%. How many tons of plastic packaging should the company aim to recover each year to meet the EPR requirement?",
        "answer": "EPR Recovery Target Calculation: To meet the recovery target of 80%, the company should aim to recover 80% 80% of 500 tons, which is: Recovery Target=80%×500 tons=0.80×500 tons=400 tons Recovery Target=80%×500tons=0.80×500tons=400tons So, the company should aim to recover 400 tons of plastic packaging each year."
    },
    {
        "context": content_2,
        "question": "What is the report quarter and when did it end?",
        "answer": "The reported quarter is the first financial quarter of 2023. The quarter ended on April 29, 2023"
    },
    {
        "context": content_3,
        "question": "If a municipality introduces a pay-as-you-throw program as part of its EPR strategy, charging households $0.05 per pound of collected waste, calculate the cost for a household that generates 30 pounds per week.",
        "answer": "To calculate the weekly cost for a household generating 30 pounds of waste: Cost = $0.05/pound × 30 pounds = $1.50 Thus, the household would incur a cost of $1.50 per week for waste disposal under the pay-as-you-throw program.."
    },
    {
        "context": content_4,
        "question": "What's the OP margin for the current quarter?",
        "answer": "The GAAP operating margin for the current quarter was -0.3%. After adjusting for specific non-recurring items, the operating margin improved to 0.5%."
    },
    {
        "context": content_5,
        "question": "What were Gap Inc's net sales for the first quarter of fiscal 2023 and how did they compare to the previous year?",
        "answer": "Gap Inc. reported net sales of $3.28 billion for the first quarter of fiscal 2023, which represents a 6% decline compared to the previous year. This decline was partly due to the sale of Gap China and foreign exchange headwinds."
    },
    {
        "context": content_6,
        "question": "What is the company's outlook on net sales for the second quarter and the full fiscal year 2023?",
        "answer": "For the second quarter of fiscal 2023, Gap Inc. estimates that net sales may decrease in the mid to high-single digit range compared to the previous year's second quarter net sales of $3.86 billion. For the full fiscal year 2023, the company anticipates net sales could decrease in the low to mid-single digit range compared to the previous year's net sales of $15.6 billion."
    },
    {
        "context": content_7,
        "question": "How much cash and cash equivalents did Gap, Inc. have at the end of the quarter?",
        "answer": "Gap Inc. ended the first quarter of fiscal 2023 with cash and cash equivalents totaling $1.2 billion, which represents an increase of 38% from the prior year."
    },
    {
        "context": content_8,
        "question": "How does the company expect gross margin to change in Q2 and the full year compared to 2022?",
        "answer": "Gap Inc. expects to see an expansion in gross margin for both the second quarter and the full fiscal year 2023 compared to 2022. This anticipated improvement reflects ongoing operational efficiencies and reduced cost pressures."
    },
    {
        "context": content_9,
        "question": "Did Gap Inc. pay a dividend in the first quarter, and what is their dividend strategy going forward?",
        "answer": "Yes, Gap Inc. paid a dividend in the first quarter, with a payment of $0.15 per share totaling $55 million. The Board of Directors has approved a dividend of $0.15 per share for the second quarter of fiscal 2023 as well, indicating a consistent dividend strategy moving forward."
    },
    {
        "context": content_10,
        "question": "Did Gap Inc. have an operating profit or loss, and if so, what was the amount?",
        "answer": "Gap Inc. reported an operating loss of $10 million for the quarter, as per the GAAP metrics. The operating margin was negative 0.3%."
    }
]

formatted_data = []
for item in data:
    formatted_item = {
        "messages": [
            {"role": "system", "content": item["context"]},
            {"role": "user", "content": item["question"]},
            {"role": "assistant", "content": item["answer"]}
        ]
    }
    formatted_data.append(formatted_item)

# Saving the formatted data to a JSONL file
with open('/content/drive/Shareddrives/Data255_Lab/Lab2/part3/qa_data.jsonl', 'w') as outfile:
    for entry in formatted_data:
        json.dump(entry, outfile)
        outfile.write('\n')

In [ ]:
path = '/content/drive/Shareddrives/Data255_Lab/Lab2/part3/qa_data.jsonl'

with open(path, 'r') as file:
    for line in file:
        record = json.loads(line)
        print(record)

{'messages': [{'role': 'system', 'content': 'managing inventory and integrating pack and hold inventory in fiscal 2023; expected ending inventory in \nfiscal 2023; expected cash flow trends through fiscal 2023; paying down our asset -backed line of credit \nin fiscal 2023 ; our dividend strategy; the expected impact of the Gap China transition, foreign exchange \nheadwinds and an additional fiscal week on second quarter and fiscal 2023 net sales; expected second \nquarter and fiscal 2023 net sales; expected second quarter and f iscal 2023 adjusted gross margin; \nexpected gross margin improvement in fiscal 2023; expected air freight expense in the second quarter and fiscal 2023; expected inflationary impacts, including commodity costs and ocean freight rates, in the second quarter and fiscal 2023; expected inventory positions  and promotional  activity in the second \nquarter and fiscal 2023; expected ROD in the second quarter and fiscal 2023; expected adjusted\non our financial condit

In [ ]:
client = OpenAI(api_key = OPENAI_API_KEY)

In [ ]:
client.files.create(
    file=Path("/content/drive/Shareddrives/Data255_Lab/Lab2/part3/qa_data.jsonl"),
    purpose="fine-tune",
)

FileObject(id='file-aNDnTpZzT2I0vrSIDJ5Yfl3u', bytes=42789, created_at=1713918057, filename='qa_data.jsonl', object='file', purpose='fine-tune', status='processed', status_details=None)

In [ ]:
try:
    client.fine_tuning.jobs.create(
        model="gpt-3.5-turbo",
        training_file="file-aNDnTpZzT2I0vrSIDJ5Yfl3u",
    )
except openai.APIConnectionError as e:
    print("The server could not be reached")
    print(e.__cause__)  # an underlying Exception, likely raised within httpx.
except openai.RateLimitError as e:
    print("A 429 status code was received; we should back off a bit.")
except openai.APIStatusError as e:
    print("Another non-200-range status code was received")
    print(e.status_code)
    print(e.response)

In [ ]:
# List 1 fine-tuning jobs
client.fine_tuning.jobs.list(limit=1)

SyncCursorPage[FineTuningJob](data=[FineTuningJob(id='ftjob-XS30cb2tbB4rhG3yrOzZXFR6', created_at=1713918066, error=Error(code=None, message=None, param=None, error=None), fine_tuned_model='ft:gpt-3.5-turbo-0125:personal::9HKlwuuf', finished_at=1713918451, hyperparameters=Hyperparameters(n_epochs=10, batch_size=1, learning_rate_multiplier=2), model='gpt-3.5-turbo-0125', object='fine_tuning.job', organization_id='org-o0KWL7PVTECPL0Oa0Xhu5lzf', result_files=['file-3YYLVKQeVHK4x07rfFGfOfEG'], seed=1882576823, status='succeeded', trained_tokens=93280, training_file='file-aNDnTpZzT2I0vrSIDJ5Yfl3u', validation_file=None, integrations=[], user_provided_suffix=None)], object='list', has_more=True)

In [ ]:
# Get output from fine-tuned model
completion = client.chat.completions.create(
  model="ft:gpt-3.5-turbo-0125:personal::9HKlwuuf",
  messages=[
      {"role": "system", "content": content_2},
      {"role": "user", "content": "What is the report quarter and when did it end?"}
  ]
)

completion.choices[0].message

ChatCompletionMessage(content='The reported quarter is the first financial quarter of 2023. The quarter ended on April 29, 2023', role='assistant', function_call=None, tool_calls=None)

In [ ]:
# Get output from fine-tuned model
completion = client.chat.completions.create(
  model="ft:gpt-3.5-turbo-0125:personal::9HKlwuuf",
  messages=[
      {"role": "system", "content": content_1},
      {"role": "user", "content": "A company produces 500 tons of plastic packaging annually and is subject to an Extended Producer Responsibility (EPR) scheme with a recovery target of 80%. How many tons of plastic packaging should the company aim to recover each year to meet the EPR requirement?"}
  ]
)

completion.choices[0].message

ChatCompletionMessage(content='The company should aim to recover 80% of 500 tons, which is: Recovery target = 80% of 500 tons = 0.80 × 500 tons = 400 tons So, the company should aim to recover 400 tons of plastic packaging each year.', role='assistant', function_call=None, tool_calls=None)

In [ ]:
predictions = []

predictions.append("The reported quarter is the first financial quarter of 2023. The quarter ended on April 29, 2023")
predictions.append("The company should aim to recover 80% of 500 tons, which is: Recovery target = 80% of 500 tons = 0.80 × 500 tons = 400 tons So, the company should aim to recover 400 tons of plastic packaging each year.")
predictions

['The reported quarter is the first financial quarter of 2023. The quarter ended on April 29, 2023',
 'The company should aim to recover 80% of 500 tons, which is: Recovery target = 80% of 500 tons = 0.80 × 500 tons = 400 tons So, the company should aim to recover 400 tons of plastic packaging each year.']

In [ ]:
# Load the ROUGE metric
rouge = load_metric('rouge')

references = [
    ["The reported quarter is the first financial quarter of 2023. The quarter ended on April 29, 2023"],
    ["EPR Recovery Target Calculation: To meet the recovery target of 80%, the company should aim to recover 80% 80% of 500 tons, which is: Recovery Target=80%×500 tons=0.80×500 tons=400 tons Recovery Target=80%×500tons=0.80×500tons=400tons So, the company should aim to recover 400 tons of plastic packaging each year."]
]

# Compute ROUGE scores
results = rouge.compute(predictions=predictions, references=references)
results

/usr/local/lib/python3.10/dist-packages/datasets/load.py:759: FutureWarning: The repository for rouge contains custom code which must be executed to correctly load the metric. You can inspect the repository content at https://raw.githubusercontent.com/huggingface/datasets/2.19.0/metrics/rouge/rouge.py
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this metric from the next major release of `datasets`.
  warnings.warn(


{'rouge1': AggregateScore(low=Score(precision=1.0, recall=0.6666666666666666, fmeasure=0.8), mid=Score(precision=1.0, recall=0.8333333333333333, fmeasure=0.9), high=Score(precision=1.0, recall=1.0, fmeasure=1.0)),
 'rouge2': AggregateScore(low=Score(precision=0.918918918918919, recall=0.6071428571428571, fmeasure=0.7311827956989247), mid=Score(precision=0.9594594594594594, recall=0.8035714285714286, fmeasure=0.8655913978494624), high=Score(precision=1.0, recall=1.0, fmeasure=1.0)),
 'rougeL': AggregateScore(low=Score(precision=0.9736842105263158, recall=0.6491228070175439, fmeasure=0.7789473684210527), mid=Score(precision=0.986842105263158, recall=0.8245614035087719, fmeasure=0.8894736842105264), high=Score(precision=1.0, recall=1.0, fmeasure=1.0)),
 'rougeLsum': AggregateScore(low=Score(precision=0.9736842105263158, recall=0.6491228070175439, fmeasure=0.7789473684210527), mid=Score(precision=0.986842105263158, recall=0.8245614035087719, fmeasure=0.8894736842105264), high=Score(precisi

In [ ]:
bleu = evaluate.load('bleu')

references = [
    ["The reported quarter is the first financial quarter of 2023. The quarter ended on April 29, 2023"],
    ["EPR Recovery Target Calculation: To meet the recovery target of 80%, the company should aim to recover 80% 80% of 500 tons, which is: Recovery Target=80%×500 tons=0.80×500 tons=400 tons Recovery Target=80%×500tons=0.80×500tons=400tons So, the company should aim to recover 400 tons of plastic packaging each year."]
]

results = bleu.compute(predictions=predictions, references=references,
          max_order = 2)
results

{'bleu': 0.7563892245243717,
 'precisions': [0.9333333333333333, 0.8904109589041096],
 'brevity_penalty': 0.8297202637161937,
 'length_ratio': 0.8426966292134831,
 'translation_length': 75,
 'reference_length': 89}